In [1]:
import pandas as pd

In [8]:
atf2_file = "/data5/deepro/starrseq/papers/results/6_link_da_enhancers_to_de_genes/data/da_de_peaks/ATF2/direct_loss.bed"
lef1_file = "/data5/deepro/starrseq/papers/results/6_link_da_enhancers_to_de_genes/data/da_de_peaks/LEF1/direct_loss.bed"

In [9]:
def read_tf_df(filename, libn):
    colnames = ["chrom", "start", "end", "chrom_coord", f"{libn}_act", "strand", f"{libn}_log2FoldChange_act", f"gene_name", f"{libn}_log2FoldChange_exp", f"{libn}_padj_act", f"{libn}_padj_exp", "CC_peak", f"{libn}_peak"]
    return pd.read_csv(filename, sep="\t", header=None, names=colnames)

In [10]:
atf2_df = read_tf_df(atf2_file, "ATF2")
lef1_df = read_tf_df(lef1_file, "LEF1")

In [24]:
atf2_regions = atf2_df.loc[:, ["chrom_coord", "gene_name"]].apply(lambda x: "_".join(x), axis=1).values
lef1_regions = lef1_df.loc[:, ["chrom_coord", "gene_name"]].apply(lambda x: "_".join(x), axis=1).values

In [27]:
set(atf2_regions).intersection(set(lef1_regions))

{'chr10_113067322_113067822_TCF7L2',
 'chr10_133244815_133245315_FUOM',
 'chr10_133244815_133245315_SCART1',
 'chr10_133244915_133245415_FUOM',
 'chr10_133244915_133245415_KNDC1',
 'chr10_133244915_133245415_SCART1',
 'chr12_68787313_68787813_SLC35E3',
 'chr13_40660115_40660615_FOXO1',
 'chr16_85562401_85562901_GSE1',
 'chr16_86502199_86502672_FENDRR',
 'chr3_152379218_152379718_MBNL1',
 'chr3_152379318_152379775_MBNL1',
 'chr7_104943194_104943694_KMT2E',
 'chr7_104943294_104943794_KMT2E',
 'chr7_43577594_43578094_STK17A',
 'chr7_43577644_43578116_STK17A'}

In [26]:
lef1_regions

array(['chr3_152379318_152379775_MBNL1', 'chr3_152379218_152379718_MBNL1',
       'chrX_46963496_46963996_ENSG00000288759',
       'chr10_133244915_133245415_KNDC1',
       'chr10_133244915_133245415_FUOM', 'chr10_133244815_133245315_FUOM',
       'chr10_133244915_133245415_SCART1',
       'chr10_133244815_133245315_SCART1',
       'chrX_46963496_46963996_ZNF674', 'chr10_133244915_133245415_MTG1',
       'chr10_133244815_133245315_MTG1', 'chr7_104943194_104943694_KMT2E',
       'chr7_104943294_104943794_KMT2E', 'chr6_680924_681424_EXOC2',
       'chr6_583381_583881_EXOC2', 'chr14_70010786_70011286_SMOC1',
       'chr14_70010886_70011386_SMOC1', 'chr14_70011186_70011686_SMOC1',
       'chr14_70011086_70011586_SMOC1', 'chr14_70011286_70011771_SMOC1',
       'chr14_70010986_70011486_SMOC1', 'chr6_582920_583386_EXOC2',
       'chr7_5525977_5526477_ACTB', 'chr7_5525877_5526377_ACTB',
       'chr7_5526077_5526577_ACTB', 'chr7_5525777_5526277_ACTB',
       'chr16_86502199_86502672_FENDRR', 'c